In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'ML/KRembedding/'

LIB_PATH = 'lib'

%cd drive/My\ Drive/$FOLDERNAME/$LIB_PATH

Mounted at /content/drive
/content/drive/My Drive/CS/KRembedding/lib


In [ ]:
!bash ../preprocess.sh dump-raw-wiki

download ko-wikipedia...
--2021-05-23 08:23:00--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 755247330 (720M) [application/octet-stream]
Saving to: ‘/notebooks/embedding/data/raw/kowiki-latest-pages-articles.xml.bz2’

kowiki-latest-pages 100%[===================>] 720.26M  4.63MB/s    in 2m 35s  

2021-05-23 08:25:35 (4.65 MB/s) - ‘/notebooks/embedding/data/raw/kowiki-latest-pages-articles.xml.bz2’ saved [755247330/755247330]



In [ ]:
import re
from gensim.utils import to_unicode

WIKI_REMOVE_CHARS = re.compile("'+|(=+.{2,30}=+)|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):|\n", re.UNICODE)
WIKI_SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
WIKI_REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)


def tokenize(content, token_min_len=2, token_max_len=100, lower=True):
    content = re.sub(EMAIL_PATTERN, ' ', content)  # remove email pattern
    content = re.sub(URL_PATTERN, ' ', content) # remove url pattern
    content = re.sub(WIKI_REMOVE_CHARS, ' ', content)  # remove unnecessary chars
    content = re.sub(WIKI_SPACE_CHARS, ' ', content)
    content = re.sub(MULTIPLE_SPACES, ' ', content)
    tokens = content.replace(", )", "").split(" ")

    result = [to_unicode(re.sub(WIKI_REMOVE_TOKEN_CHARS, '', token)) for token in tokens if not token.startswith('_') and token != '']
    return result

In [ ]:
from gensim.corpora import WikiCorpus, Dictionary
from gensim.utils import to_unicode

in_f = '/notebooks/embedding/data/raw/kowiki-latest-pages-articles.xml.bz2'
out_f = '/notebooks/embedding/data/processed/processed_wiki_ko.txt'
with open(out_f, 'w') as f:
  wiki = WikiCorpus(in_f, tokenizer_func=tokenize, dictionary=Dictionary())
  for i, text in enumerate(wiki.get_texts()):
    f.write(' '.join(text) + '\n')
    if i%10000 == 0: print(f'Processed {str(i)} articles')
print('Processing complete!')

Processed 0 articles


Process InputQueue-38:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/dist-packages/gensim/utils.py", line 1208, in run
    self.q.put(wrapped_chunk.pop(), block=True)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 82, in put
    if not self._sem.acquire(block, timeout):
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
!bash ../preprocess.sh dump-raw-korquad

download KorQuAD data...
--2021-05-23 11:32:27--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘/notebooks/embedding/data/raw/KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  41.5MB/s    in 0.9s    

2021-05-23 11:32:28 (41.5 MB/s) - ‘/notebooks/embedding/data/raw/KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2021-05-23 11:32:28--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) 

In [ ]:
import json
corpus_fname = '/notebooks/embedding/data/raw/KorQuAD_v1.0_train.json'
output_fname = '/notebooks/embedding/data/processed/processed_korquad_train.txt'
with open(corpus_fname) as f1, open(output_fname, 'w', encoding='utf-8') as f2:
  dataset_json = json.load(f1)
  dataset = dataset_json['data']
  for article in dataset:
    w_lines = []
    for paragraph in article['paragraphs']:
      w_lines.append(paragraph['context'])
      w_lines += [ qa['question'] + ' ' + answer['text'] for qa in paragraph['qas'] for answer in qa['answers']]
    for line in w_lines:
      f2.writelines(line + '\n')

In [ ]:
!bash ../preprocess.sh dump-raw-nsmc

download naver movie corpus...
--2021-05-23 12:09:25--  https://github.com/e9t/nsmc/raw/master/ratings.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [following]
--2021-05-23 12:09:25--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘/notebooks/embedding/data/raw/ratings.txt’

ratings.txt         100%[===================>]  18.61M  28.6MB/s    in 0.7s    

2021-05-23 12:09:26 (28.6 MB/s) - ‘/notebooks/embedding/data/raw/ratings.txt’ saved [19515078/19515078]

--

In [ ]:
corpus_fname = '/notebooks/embedding/data/raw/ratings.txt'
output_fname = '/notebooks/embedding/data/processed/processed_ratings.txt'
with_label = False

with open(corpus_fname, 'r', encoding='utf-8') as f1, open(output_fname, 'w', encoding='utf-8') as f2:
  next(f1)
  if with_label:
    for line in f1:
      f2.writelines(line.strip().split('\t')[1:].join('\u241E') + '\n')
  else:
    for line in f1:
      f2.writelines(line.strip().split('\t')[1] + '\n')